FINAL PROJECT: BEHAVIORIAL CLONING WOOHOO

Collaborators:
- Nicole Wong
- Audrey Li
- Elaine Pham

# Working Simulation

Features:

1. Simulation contains foam brick, iiwa, and floor

undefined. Commands iiwa using teleop 

undefined. Manually prints joint angles and corresponding end effector translation

This code comes from the textbook example (https://deepnote.com/workspace/hao-pham-efb6-9e45732a-afda-453c-8485-472c524bf503/project/1-Introduction-Duplicate-7231f625-f3dc-4607-8336-24da84aacfc0/notebook/intro-58422166dd0b4e17bb10cf8ad3d886e4)

No significant changes made.

In [4]:
import logging
import numpy as np
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from manipulation import running_as_notebook
from manipulation.meshcat_utils import MeshcatPoseSliders, WsgButton
from manipulation.scenarios import (AddIiwaDifferentialIK,
                                    MakeManipulationStation)

from pydrake.all import  (Box, SpatialInertia, UnitInertia, CoulombFriction)

# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://a569df8b-a69d-4307-8cbd-4f24b608e3b4.deepnoteproject.com/7001/


In [5]:
def MakeTeleopAndSimulate():
    model_directives = """
    directives:
    - add_directives:
        file: package://manipulation/iiwa_and_wsg.dmd.yaml
    - add_model:
        name: foam_brick
        file: package://drake/examples/manipulation_station/models/061_foam_brick.sdf
        default_free_body_pose:
            base_link:
                translation: [0, -0.6, 0]
    - add_model:
        name: floor
        file: package://manipulation/floor.sdf
    - add_weld:
        parent: world
        child: floor::box
        X_PC:
            translation: [0, 0, 0]
    """
    builder = DiagramBuilder()

    time_step = 0.001
    station = builder.AddSystem(
        MakeManipulationStation(model_directives, time_step=time_step, 
        # prefinalize_callback=AddBox))
        ))
    plant_robot = station.GetSubsystemByName("plant")
    iiwa = plant_robot.GetModelInstanceByName("iiwa")
    controller_plant = station.GetSubsystemByName(
        "iiwa_controller").get_multibody_plant_for_control()

    # Add a meshcat visualizer.
    meshcat.ResetRenderMode()
    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)

    # Set up differential inverse kinematics.
    differential_ik = AddIiwaDifferentialIK(
        builder,
        controller_plant,
        frame=controller_plant.GetFrameByName("iiwa_link_7"))
    builder.Connect(differential_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))
    builder.Connect(station.GetOutputPort("iiwa_state_estimated"),
                    differential_ik.GetInputPort("robot_state"))

    # Set up teleop widgets.
    teleop = builder.AddSystem(
        MeshcatPoseSliders(
            meshcat,
            min_range=MeshcatPoseSliders.MinRange(roll=0,
                                                    pitch=-0.5,
                                                    yaw=-np.pi,
                                                    x=-0.6,
                                                    y=-0.8,
                                                    z=0.0),
            max_range=MeshcatPoseSliders.MaxRange(roll=2 * np.pi,
                                                    pitch=np.pi,
                                                    yaw=np.pi,
                                                    x=0.8,
                                                    y=0.3,
                                                    z=1.1),
            body_index=plant_robot.GetBodyByName("iiwa_link_7").index()))
    builder.Connect(teleop.get_output_port(0),
                    differential_ik.get_input_port(0))
    builder.Connect(station.GetOutputPort("body_poses"),
                    teleop.GetInputPort("body_poses"))
    wsg_teleop = builder.AddSystem(WsgButton(meshcat))
    builder.Connect(wsg_teleop.get_output_port(0),
                    station.GetInputPort("wsg_position"))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    # context = diagram.CreateDefaultContext()
    # default_context = diagram.CreateDefaultContext()
    # plant_context = plant_robot.GetMyContextFromRoot(default_context)
    context = simulator.get_mutable_context()
    plant_context = plant_robot.GetMyContextFromRoot(context)
    station_context = station.GetMyContextFromRoot(context)
    q0 = np.array([ 1.40666193e-05,  1.56461165e-01, -3.82761069e-05,
                        -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])
    # set the joint positions of the kuka arm
    plant_robot.SetPositions(plant_context, iiwa, q0)


    old_pose = np.array([0,0,0])
    old_angles = np.array([0,0,0,0,0,0,0])
    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
            # # joint angles
            # print(station.GetOutputPort("iiwa_position_measured").Eval(station_context))
            gripper_ind = plant_robot.GetBodyIndices(iiwa)[-1]
            gripper = plant_robot.get_body(gripper_ind)
            pose = gripper.EvalPoseInWorld(plant_context).translation()
            angles = plant_robot.GetPositions(plant_context, iiwa)
            # iiwa moved significantly
            if np.any(np.abs(pose-old_pose) > 1e-5):
                print('old pose', old_pose)
                print('end effector ', pose)
                print('iiwa joint angles', angles)
            old_pose = pose
            old_angles = angles
        meshcat.DeleteButton("Stop Simulation")
    else:
        simulator.AdvanceTo(10)

MakeTeleopAndSimulate()

Keyboard Controls:
roll : KeyQ / KeyE
pitch : KeyW / KeyS
yaw : KeyA / KeyD
x : KeyJ / KeyL
y : KeyI / KeyK
z : KeyO / KeyU
Press Space to open/close the gripper
Press Escape to stop the simulation
old pose [0 0 0]
end effector  [ 0.46082949 -0.00432446  0.69871325]
iiwa joint angles [ 0.44851216  0.2365803  -0.53941687 -1.64914558 -2.98934665  1.91052788
  0.12412552]
old pose [ 0.46082949 -0.00432446  0.69871325]
end effector  [ 0.46058542 -0.00326264  0.69926448]
iiwa joint angles [ 0.43312683  0.23284433 -0.51632986 -1.64897216 -2.99096373  1.90929125
  0.1163264 ]
old pose [ 0.46058542 -0.00326264  0.69926448]
end effector  [ 0.46057994 -0.00323914  0.6992718 ]
iiwa joint angles [ 0.43280018  0.23277629 -0.51589759 -1.64897778 -2.99126941  1.90930469
  0.11614111]
old pose [ 0.46057994 -0.00323914  0.6992718 ]
end effector  [ 0.46057445 -0.00321577  0.6992791 ]
iiwa joint angles [ 0.43247583  0.23270868 -0.51546832 -1.64898336 -2.99157302  1.90931806
  0.11595707]
old pose [ 0.460

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a569df8b-a69d-4307-8cbd-4f24b608e3b4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>